In [1]:
from PIL import Image
import glob
import numpy as np
from collections import defaultdict
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img
from keras.models import Model
from keras.layers import Dense, MaxPooling2D, Input, Flatten, Dropout
from keras.applications.mobilenet import preprocess_input
from keras.applications import MobileNet
from keras import backend as K
import matplotlib.pylab as plt
import cv2
import os, shutil

from shutil import copyfile

C:\Users\ADETOLA\Anaconda3\envs\camsparknn\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# # Train-test split, only run this code once
# INPUT = 'Org'
# VAL_NUMBER = 0.25
    
# def get_files(path):
#     dirs = [x[0] for x in os.walk(path)]
#     files = {}
#     for directory in dirs:
#         classes = os.path.basename(os.path.normpath(directory))
#         files[classes] = glob.glob(os.path.join(directory, '*.jpg'))
 
#     return files

# all_files = get_files(INPUT)
# new_dict = {k:v for k,v in all_files.items() if v}

# train_dir = 'Train/'
# test_dir = 'Test/'
# train_dirs = os.listdir(train_dir)
# test_dirs = os.listdir(test_dir)

# for _class in new_dict.keys():
#     files = new_dict[_class]
#     np.random.shuffle(files)
#     file_size = len(files)
#     val_files = files[:int(file_size * VAL_NUMBER)]
    
#     for file in files:
#         if file in val_files:
#                 basename = os.path.basename(file)
# #                 new_name = _class+ '_' + basename
# #                 os.rename(file, file.replace(basename, new_name))
#                 shutil.move('Org/'+_class+'/'+basename, test_dir+'/'+_class)
#         else:
#                 basename = os.path.basename(file)
# #                 new_name = _class+ '_' + basename
# #                 os.rename(file, file.replace(basename, new_name))
#                 shutil.move('Org/'+_class+'/'+basename, train_dir+'/'+_class)


In [3]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_dir = 'Train/'
test_dir = 'Test/'
IM_WIDTH = 224 
IM_HEIGHT = 224
batch_size = 30

train_generator = train_datagen.flow_from_directory(
        train_dir, 
        target_size=(IM_WIDTH, IM_HEIGHT), 
        batch_size=batch_size,
        seed=1234)

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(IM_WIDTH, IM_HEIGHT),
        batch_size=batch_size,
        seed=1234
        )

Found 2148 images belonging to 3 classes.
Found 715 images belonging to 3 classes.


In [4]:
def get_nb_files(directory):
    """Get number of files by searching directory recursively"""
    if not os.path.exists(directory):
        return 0
    cnt = 0
    for r, dirs, files in os.walk(directory):
        for dr in dirs:
            cnt += len(glob.glob(os.path.join(r, dr + "/*")))
    return cnt

In [5]:
def add_new_last_layer(base_model, nb_classes):

    """Add last layer to the convnet
       Args:
        base_model: keras model excluding top
        nb_classes: # of classes
    Returns:
        new keras model with last layer
    """
    x = base_model.output
    x = MaxPooling2D((3, 3), border_mode='same', strides=1, name='max_pool')(x)
    x = Dropout(0.2, seed=1234)(x)
    x = Flatten()(x)
#     x = Dense(100, activation='relu', name='fc1')(x)
    predictions = Dense(3, activation='softmax')(x)
    model = Model(input=base_model.input, output=predictions)
    return model


In [6]:
def setup_to_transfer_learn(model, base_model):
    """Freeze all layers and compile the model"""
    for layer in base_model.layers:
        layer.trainable = False
    model.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

In [7]:
nb_classes = len(glob.glob(train_dir + "/*"))
nb_classes
nb_train_samples = get_nb_files(train_dir)
nb_train_samples
nb_test_samples = get_nb_files(test_dir)
nb_test_samples

3

2148

715

In [8]:
if(K.image_dim_ordering() == 'th'):
    input_tensor = Input(shape=(3, 224, 224))
else:
    input_tensor = Input(shape=(224, 224, 3))

In [9]:
base_model = MobileNet(input_tensor = input_tensor, input_shape=(224, 224, 3), weights='imagenet', include_top=False)
model = add_new_last_layer(base_model, nb_classes)
# base_model.summary()

C:\Users\ADETOLA\Anaconda3\envs\camsparknn\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D((3, 3), strides=1, name="max_pool", padding="same")`
  # This is added back by InteractiveShellApp.init_path()
C:\Users\ADETOLA\Anaconda3\envs\camsparknn\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  app.launch_new_instance()


In [10]:
train_generator.image_shape
test_generator.image_shape

(224, 224, 3)

(224, 224, 3)

In [ ]:
setup_to_transfer_learn(model, base_model)
history_tl = model.fit_generator(train_generator,
                                   samples_per_epoch=nb_train_samples,
                                 nb_epoch=1,
                                   validation_data=test_generator,
                                )
#                                   nb_val_samples=nb_test_samples) 



C:\Users\ADETOLA\Anaconda3\envs\camsparknn\lib\site-packages\ipykernel_launcher.py:5: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  """
C:\Users\ADETOLA\Anaconda3\envs\camsparknn\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=<keras.pre..., steps_per_epoch=71, epochs=1)`
  """


Epoch 1/1
 4/71 [>.............................] - ETA: 17:27 - loss: 6.1786 - acc: 0.6167

In [ ]:
model.save(output_model_file)

In [ ]:
def plot_training(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(acc))
    plt.plot(epochs, acc, 'r.')
    plt.plot(epochs, val_acc, 'r')
    plt.title('Training and validation accuracy')
    plt.savefig('accuracy.png')
    plt.figure()
    plt.plot(epochs, loss, 'r.')
    plt.plot(epochs, val_loss, 'r-')
    plt.title('Training and validation loss')
    plt.savefig('loss.png')

In [ ]:
plot_training(history_tl)